In [1]:
#Install the necessary modules.
import pandas as pd
from thunderpack import ThunderReader  
import pandas as pd
import random
import zipfile
import io
import tempfile
from tqdm import tqdm

In [2]:
#This part is importing the metadata into dfs
df_0 = pd.read_csv('/media/gregory178/Thunderpacks/Dropbox/zz_EHR_Thunderpacks/MGB/MGB_Deidentified_Notes_March12th2024/mgb_notes_2016_metadata.csv')
df_1 = pd.read_csv('/media/gregory178/Thunderpacks/Dropbox/zz_EHR_Thunderpacks/MGB/MGB_Deidentified_Notes_March12th2024/mgb_notes_2017_metadata.csv')
df_2 = pd.read_csv('/media/gregory178/Thunderpacks/Dropbox/zz_EHR_Thunderpacks/MGB/MGB_Deidentified_Notes_March12th2024/mgb_notes_2018_metadata.csv')
df_3 = pd.read_csv('/media/gregory178/Thunderpacks/Dropbox/zz_EHR_Thunderpacks/MGB/MGB_Deidentified_Notes_March12th2024/mgb_notes_2019_metadata.csv')
df_4 = pd.read_csv('/media/gregory178/Thunderpacks/Dropbox/zz_EHR_Thunderpacks/MGB/MGB_Deidentified_Notes_March12th2024/mgb_notes_2020_metadata.csv')
df_5 = pd.read_csv('/media/gregory178/Thunderpacks/Dropbox/zz_EHR_Thunderpacks/MGB/MGB_Deidentified_Notes_March12th2024/mgb_notes_2021_metadata.csv')
df_6 = pd.read_csv('/media/gregory178/Thunderpacks/Dropbox/zz_EHR_Thunderpacks/MGB/MGB_Deidentified_Notes_March12th2024/mgb_notes_2022_metadata.csv')

dfs = pd.concat([df_0,df_1, df_2, df_3, df_4, df_5, df_6], ignore_index=True)  
print(len(dfs))
dfs.head()

22601041


,BDSPPatientID,ContactDate,InpatientNoteTypeDSC,DeidentifiedName
0,118139267,20160218,Progress Notes,Notes_13246153407_1072726330_20160218.txt
1,120478057,20160829,Progress Notes,Notes_13333565700_1075065164_20160829.txt
2,117769902,20161017,Discharge Summary,Notes_13293992978_1072357331_20161017.txt
3,122117332,20160218,MR AVS Snapshot,Notes_13280349751_1076704882_20160218.txt
4,115708902,20160604,ED Notes,Notes_13255380096_1070297079_20160604.txt


In [3]:
#Data filtering of metadata notes.
#Changing date to datetime format.
dfs['ContactDate'] = pd.to_datetime(dfs['ContactDate'], format='%Y%m%d')

#The following dates are Epic dates only for MGB.
start_date = pd.Timestamp('2016-05-01')
end_date = pd.Timestamp('2023-01-01') 
complete_df_n = dfs[(dfs['ContactDate'] >= start_date) & (dfs['ContactDate'] < end_date)].reset_index(drop=True)

#Filter all notes for discharge notes only for MGB.
complete_df_discharge = complete_df_n[complete_df_n.InpatientNoteTypeDSC=='Discharge Summary'].reset_index(drop=True)
complete_df_discharge.head()
complete_df_discharge.to_csv("complete_df_discharge.csv", index=False)
complete_df_discharge=pd.read_csv("complete_df_discharge.csv")


In [4]:
#Make sure that ShiftedContactDTS is in date time format.
icdPlus = pd.read_csv('/home/gregory178/Desktop/NAX project/NAX_SDH/patientIDs_ICD_plus_SDH_MGB.csv')
icdPlus['ShiftedContactDTS'] = pd.to_datetime(icdPlus['ShiftedContactDTS'], format='%Y-%m-%d %H:%M:%S.%f').dt.date
icdPlus['ShiftedContactDTS'] = pd.to_datetime(icdPlus['ShiftedContactDTS'])
icdPlus['BDSPPatientID'] = icdPlus['BDSPPatientID'].astype('int64')

#Merge the data.
final_cpt_pos_notes = pd.merge(complete_df_discharge, icdPlus, on=['BDSPPatientID'], how='inner')
print(final_cpt_pos_notes[['ContactDate', 'ShiftedContactDTS']].dtypes)

#Ensure ContactDate is in date time format.
final_cpt_pos_notes['ContactDate'] = pd.to_datetime(final_cpt_pos_notes['ContactDate'])

#Filter for icd's assigned within 30d of ContactDate. 
date_difference = abs((final_cpt_pos_notes['ContactDate'] - final_cpt_pos_notes['ShiftedContactDTS']).dt.days)

# Filter the DataFrame to include only rows where the difference is less than 30 days.
ICDPositiveDS = final_cpt_pos_notes[date_difference < 30].reset_index(drop=True)
ICDPositiveDS.to_csv("ICD_Postive_DS_before_500.csv", index=False)
ICDPositiveDS = pd.read_csv('/home/gregory178/Desktop/NAX project/NAX_SDH/ICD_Postive_DS_before_500.csv')

ContactDate                  object
ShiftedContactDTS    datetime64[ns]
dtype: object


In [5]:
#Create the CPT Positive List
reader = ThunderReader('/media/gregory178/Thunderpacks/Dropbox/zz_EHR_Thunderpacks/MGB/thunderpack_cpt_1m_MGB')

#CPT positive means the patient received a code indicating that the recieved a brain mri or brain ct scan. 
dfs = []

#Create a loop to loop over all keys. 
for i in tqdm(range(1, (len(list(reader.keys()))+1))):    
    df = reader[f'cpt_partition_{i}']
    df = df[df.CPT.astype(str).str.contains('^(?:70450|70460|70470|70551|70552|70553)')] 
    dfs.append(df)
df_cpt = pd.concat(dfs, axis=0, ignore_index=True)  

#Output cpt positive list to a csv and only save unique patient ids. 
df_cpt = df_cpt[['BDSPPatientID','CPT','StartDTS']].drop_duplicates(ignore_index=True)
#Save as a csv file.

df_cpt.to_csv('patientIDs_CPT_HeadMRICT_MGB.csv', index=False)    

100%|██████████| 735/735 [21:00<00:00,  1.71s/it]


In [6]:
#Change the CPT list to date time format. 
cpt = pd.read_csv('/home/gregory178/Desktop/NAX project/NAX_SDH/patientIDs_CPT_HeadMRICT_MGB.csv')
cpt['StartDTS'] = pd.to_datetime(cpt['StartDTS'], format='%Y-%m-%d %H:%M:%S.%f').dt.date
cpt['StartDTS'] = pd.to_datetime(cpt['StartDTS'])
cpt['BDSPPatientID'] = cpt['BDSPPatientID'].astype('int64')

#Merge the ICD and CPT list.
final_cpt_pos_notes = pd.merge(ICDPositiveDS, cpt, on=['BDSPPatientID'], how='inner')
final_cpt_pos_notes['ContactDate'] = pd.to_datetime(final_cpt_pos_notes['ContactDate'])
final_cpt_pos_notes['StartDTS'] = pd.to_datetime(final_cpt_pos_notes['StartDTS'])

#Filter for CPTs given within 30 days of creation. 
date_difference = abs((final_cpt_pos_notes['ContactDate'] - final_cpt_pos_notes['StartDTS']).dt.days)
ICDPositiveDS_CPT = final_cpt_pos_notes[date_difference < 30].reset_index(drop=True)

#Make it into a csv.
ICDPositiveDS_CPT.to_csv("ICD_Postive_DS_CPT_before_500.csv",index=False)

In [7]:
#Here we are defining the function that will randomly select notes and check if their word count is above 500. 
def select_random_note_zip(df, patient_id, base_zip_path):
    patient_notes = df[df['BDSPPatientID'] == patient_id]
    valid_notes = []
    zip_file = None

    for _, row in patient_notes.iterrows():
        year = row['DeidentifiedName'].split('_')[-1][:4]
        file_path = f'mgb_notes_{year}/{year}/' + row['DeidentifiedName']
        zip_path = f'{base_zip_path}mgb_notes_{year}.zip'
        try:
            if zip_file is None or zip_file.filename != zip_path:
                if zip_file is not None:
                    zip_file.close()
                zip_file = zipfile.ZipFile(zip_path, 'r')
            
            with zip_file.open(file_path) as file_obj:
                try:
                    contents = file_obj.read().decode('utf-8')
                except UnicodeDecodeError:
                    try:
                        contents = file_obj.read().decode('latin1')
                    except UnicodeDecodeError:
                        contents = file_obj.read().decode('iso-8859-1')
                #Here we sort through based on word count. 
                word_count = len(contents.split())
                print(f"Length of contents for {file_path}: {word_count}")
                if word_count >= 500:
                    valid_notes.append((row['BDSPPatientID'], contents, row['ContactDate'], word_count, row['DeidentifiedName'], 'MGB'))
        except (KeyError, FileNotFoundError, zipfile.BadZipFile) as e:
            print(f"Error processing file {file_path}: {str(e)}")
            continue
    if zip_file is not None:
        zip_file.close()
    if valid_notes:
        #Randomly select one note from notes longer than 500 words.
        selected_note = random.choice(valid_notes) 
        return selected_note
    else:
        return None

base_zip_path= '/media/gregory178/Elements/MGB notes/zip/'
ICDPL_df=pd.read_csv('/home/gregory178/Desktop/NAX project/NAX_SDH/ICD_Postive_DS_CPT_before_500.csv')
#This is the list of all patients with at least one ICD+ code for MGB.
FULLICDPL_list = ICDPL_df['BDSPPatientID'].tolist() 

columns = ['BDSPPatientID', 'note_text', 'CreateDate', 'length', 'deidentifiedname', 'hospital']

#Making ICD + and - groups for MGB.
ICDPL_MGB = pd.DataFrame()
ICDMI_MGB = pd.DataFrame()



In [8]:
#Now we are doing ICD+ and discharge notes.
counterPL = 0
added_patient_idsPL = []
print(len(added_patient_idsPL))
print(added_patient_idsPL)

#To ensure the reproducibility.
random.seed(2024)

while len(ICDPL_MGB) < 750:
    randpatientID = random.choice(FULLICDPL_list) 
    UPDATED_LEN_GROUP=len(FULLICDPL_list)
    print(f"Updated length list {UPDATED_LEN_GROUP}.")
    if randpatientID not in added_patient_idsPL:
        rnote2 = select_random_note_zip(complete_df_discharge, randpatientID, base_zip_path)
        if rnote2 is not None:
            ICDPL_MGB = pd.concat([ICDPL_MGB, pd.DataFrame([rnote2])], ignore_index=True) 
            added_patient_idsPL.append(randpatientID)
            FULLICDPL_list.remove(randpatientID)
            counterPL += 1
            print(f"Added note for patient {randpatientID}. Total notes added: {counterPL}")
        else:
            print(f"No valid notes for patient {randpatientID}")
            FULLICDPL_list.remove(randpatientID)
    else:
        print(f"Patient {randpatientID} already has a note added")
        FULLICDPL_list.remove(randpatientID)
    if not FULLICDPL_list:
        print("No more patient IDs left to process")

ICDPL_MGB.columns = ['BDSPPatientID', 'NoteContent', 'ContactDate', 'WordCount', 'NoteFileName', 'Site']
ICDPL_MGB.to_csv('MGB_sampling_cohort_ICD+_discharge_notes.csv', index=False)

0
[]
Updated length list 121369.
Length of contents for mgb_notes_2019/2019/Notes_13447204985_2424773631_20190124.txt: 0
No valid notes for patient 113921671
Updated length list 121368.
Length of contents for mgb_notes_2017/2017/Notes_13308112458_1447436562_20170624.txt: 1092
Length of contents for mgb_notes_2017/2017/Notes_13308118911_1461153155_20170706.txt: 2734
Length of contents for mgb_notes_2017/2017/Notes_13317689592_1545405034_20170917.txt: 2514
Length of contents for mgb_notes_2017/2017/Notes_13319806223_1561663305_20171001.txt: 1260
Added note for patient 119842064. Total notes added: 1
Updated length list 121367.
Length of contents for mgb_notes_2017/2017/Notes_13323050109_1607827103_20170106.txt: 4151
Length of contents for mgb_notes_2017/2017/Notes_13323337605_1626976510_20170121.txt: 1931
Added note for patient 114518157. Total notes added: 2
Updated length list 121366.
Length of contents for mgb_notes_2021/2021/Notes_13628865839_6396909033_20210328.txt: 0
Length of cont

In [10]:
#Here we are creating the ICD minus sampling cohort.
icd_minus_df = pd.read_csv('patientIDs_ICD_minus2_SDH_MGB.csv')
full_patient_list = sorted( set(complete_df_discharge.BDSPPatientID) & set(icd_minus_df.BDSPPatientID) )
FULLICDMI_list = sorted( set(full_patient_list) - set(FULLICDPL_list) )  

In [11]:
#Now we are making the  ICD minus csv and discharge notes

counterMI = 0
added_patient_idsMI = []
print(len(added_patient_idsMI))
print(added_patient_idsMI)
random.seed(13)

while len(ICDMI_MGB) < 750:
    randpatientID = random.choice(FULLICDMI_list) 
    UPDATED_LEN_GROUP=len(FULLICDMI_list)
    print(f"Updated length list {UPDATED_LEN_GROUP}.")
    if randpatientID not in added_patient_idsMI:
        rnote2 = select_random_note_zip(complete_df_discharge, randpatientID, base_zip_path)
        if rnote2 is not None:
            ICDMI_MGB = pd.concat([ICDMI_MGB, pd.DataFrame([rnote2])], ignore_index=True) 
            added_patient_idsMI.append(randpatientID)
            FULLICDMI_list.remove(randpatientID)
            counterMI += 1
            print(f"Added note for patient {randpatientID}. Total notes added: {counterMI}")
        else:
            print(f"No valid notes for patient {randpatientID}")
            FULLICDMI_list.remove(randpatientID)
    else:
        print(f"Patient {randpatientID} already has a note added")
        FULLICDMI_list.remove(randpatientID)
    if not FULLICDMI_list:
        print("No more patient IDs left to process")

ICDMI_MGB.columns = ['BDSPPatientID', 'NoteContent', 'ContactDate', 'WordCount', 'NoteFileName', 'Site']
ICDMI_MGB.to_csv('MGB_sampling_cohort_ICD_minus_discharge_notes.csv', index=False)

0
[]
Updated length list 37264.
Length of contents for mgb_notes_2016/2016/Notes_13277290853_998475736_20160506.txt: 269
Length of contents for mgb_notes_2016/2016/Notes_13279973881_1021790777_20160519.txt: 263
Length of contents for mgb_notes_2017/2017/Notes_13318855192_1545875132_20170613.txt: 4038
Length of contents for mgb_notes_2017/2017/Notes_13318855192_1544542624_20170612.txt: 4022
Length of contents for mgb_notes_2018/2018/Notes_13372450903_2098836154_20180812.txt: 1947
Length of contents for mgb_notes_2021/2021/Notes_13546441773_6114886930_20210814.txt: 256
Added note for patient 116291874. Total notes added: 1
Updated length list 37263.
Length of contents for mgb_notes_2016/2016/Notes_13352426779_1384997463_20160721.txt: 2254
Added note for patient 114884810. Total notes added: 2
Updated length list 37262.
Length of contents for mgb_notes_2016/2016/Notes_13269321066_1273020171_20160626.txt: 666
Added note for patient 114130469. Total notes added: 3
Updated length list 37261.

KeyboardInterrupt: 